In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO
from google.colab import drive
import time  # Import the time module

# Mount Google Drive
drive.mount('/content/drive')

# SHEETCHANGE
# Define sheet names
TEST_SHEET_NAME = 'COHORT'
LINKS_SHEET_NAME = 'links'

# Google Drive and Sheet setup
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/My Drive/my_private_files/mysheets.json',
                                                        ['https://www.googleapis.com/auth/drive',
                                                         'https://www.googleapis.com/auth/drive.file',
                                                         'https://www.googleapis.com/auth/drive.metadata'])
client = gspread.authorize(creds)
sheet = client.open_by_url('https://docs.google.com/=sharing')

links_sheet = sheet.worksheet(LINKS_SHEET_NAME)  # Get the "links" sheet

drive_service = build('drive', 'v3', credentials=creds)

# Ensure a shared folder exists or create it
folder_name = 'Admission Letter'
response = drive_service.files().list(q=f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'").execute()
if not response['files']:
    folder_metadata = {'name': folder_name, 'mimeType': 'application/vnd.google-apps.folder'}
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    folder_id = folder.get('id')
    drive_service.permissions().create(fileId=folder_id, body={'type': 'anyone', 'role': 'reader'}, fields='id').execute()
else:
    folder_id = response['files'][0]['id']

# Ensure ID card folder exists or create it
id_card_folder_name = 'ID Card'
response_id = drive_service.files().list(q=f"name='{id_card_folder_name}' and mimeType='application/vnd.google-apps.folder'").execute()
if not response_id['files']:
    id_card_folder_metadata = {'name': id_card_folder_name, 'mimeType': 'application/vnd.google-apps.folder'}
    id_card_folder = drive_service.files().create(body=id_card_folder_metadata, fields='id').execute()
    id_card_folder_id = id_card_folder.get('id')
    drive_service.permissions().create(fileId=id_card_folder_id, body={'type': 'anyone', 'role': 'reader'}, fields='id').execute()
else:
    id_card_folder_id = response_id['files'][0]['id']

# TEMPCHANGE
# Load font and image templates
font_url = 'https://github.com/google/fonts/raw/main/ofl/poppins/Poppins-Bold.ttf'
response = requests.get(font_url)
font_file = BytesIO(response.content)
font = ImageFont.truetype(font_file, 28)
admission_template = Image.open('/content/drive/My Drive/May.png')
id_card_template = Image.open('/content/drive/My Drive/May.png')

# Process data
data = sheet.worksheet(TEST_SHEET_NAME).get_all_records()
df = pd.DataFrame(data)

batch_size = 25  # Set the batch size
for batch_start in range(0, len(df), batch_size):
    batch_end = batch_start + batch_size
    batch_df = df[batch_start:batch_end]

    for index, row in batch_df.iterrows():
        # Skip rows where links are already present
        if row.get('admission_letter') and row.get('id_card'):
            print(f"Links already present for row {index + 1}; skipping...")
            continue

        # Check for necessary data before proceeding
        if pd.isna(row['name']) or row['name'].strip() == "" or pd.isna(row['iit_guwahati_id']) or str(row['iit_guwahati_id']).strip() == "":
            print(f"Missing or empty data for row {index + 1}; skipping...")
            continue  # Skip this iteration if key data is missing or empty

        # Generate Admission Letter
        if not row.get('admission_letter'):
            admission_image_template = admission_template.copy()
            admission_draw = ImageDraw.Draw(admission_image_template)
            admission_draw.text((405, 1040), row['name'], fill='black', font=font)  # Name
            admission_draw.text((405, 1090), str(row['iit_guwahati_id']), fill='black', font=font)  # ID

            admission_filename = f"{row['name']} [{row['iit_guwahati_id']}] - Admission Letter.png"
            admission_image_path = f'/tmp/{admission_filename}'
            admission_image_template.save(admission_image_path)

            # Check if admission letter file with the same name already exists in Google Drive
            try:
                existing_admission_files = drive_service.files().list(q=f"name='{admission_filename}' and '{folder_id}' in parents", fields='files(id, modifiedTime)').execute().get('files', [])
                if existing_admission_files:
                    existing_admission_files.sort(key=lambda x: x['modifiedTime'], reverse=True)  # Sort by modified time, latest first
                    for file in existing_admission_files[1:]:  # Delete all but the most recent file
                        drive_service.files().delete(fileId=file['id']).execute()
                    admission_link = f"https://drive.google.com/file/d/{existing_admission_files[0]['id']}"
                    print(f"Found existing file for {row['name']} - Admission Letter Link: {admission_link}")
                else:
                    raise FileNotFoundError
            except FileNotFoundError:
                # Upload admission letter to Google Drive if file doesn't exist
                admission_file_metadata = {'name': admission_filename, 'parents': [folder_id]}
                admission_media = MediaFileUpload(admission_image_path, mimetype='image/png')
                admission_file = drive_service.files().create(body=admission_file_metadata, media_body=admission_media, fields='webViewLink').execute()
                admission_link = admission_file.get('webViewLink')
                print(f"Created new file for {row['name']} - Admission Letter Link: {admission_link}")

            # Update the test_sheet with the new admission letter link
            sheet.worksheet(TEST_SHEET_NAME).update_cell(index + 2, df.columns.get_loc('admission_letter') + 1, admission_link)
            print(f"Updated {TEST_SHEET_NAME} for {row['name']} with new admission letter link.")

        # Generate ID Card
        if not row.get('id_card'):
            id_card_image_template = id_card_template.copy()
            id_card_draw = ImageDraw.Draw(id_card_image_template)
            id_card_draw.text((460, 215), str(row['iit_guwahati_id']), fill='black', font=font)  # ID
            id_card_draw.text((460, 265), row['name'], fill='black', font=font)  # Name

            id_card_filename = f"{row['name']} [{row['iit_guwahati_id']}] - ID Card.png"
            id_card_image_path = f'/tmp/{id_card_filename}'
            id_card_image_template.save(id_card_image_path)

            # Check if ID card file with the same name already exists in Google Drive
            try:
                existing_id_card_files = drive_service.files().list(q=f"name='{id_card_filename}' and '{id_card_folder_id}' in parents", fields='files(id, modifiedTime)').execute().get('files', [])
                if existing_id_card_files:
                    existing_id_card_files.sort(key=lambda x: x['modifiedTime'], reverse=True)  # Sort by modified time, latest first
                    for file in existing_id_card_files[1:]:  # Delete all but the most recent file
                        drive_service.files().delete(fileId=file['id']).execute()
                    id_card_link = f"https://drive.google.com/file/d/{existing_id_card_files[0]['id']}"
                    print(f"Found existing file for {row['name']} - ID Card Link: {id_card_link}")
                else:
                    raise FileNotFoundError
            except FileNotFoundError:
                # Upload ID card to Google Drive if file doesn't exist
                id_card_file_metadata = {'name': id_card_filename, 'parents': [id_card_folder_id]}
                id_card_media = MediaFileUpload(id_card_image_path, mimetype='image/png')
                id_card_file = drive_service.files().create(body=id_card_file_metadata, media_body=id_card_media, fields='webViewLink').execute()
                id_card_link = id_card_file.get('webViewLink')
                print(f"Created new file for {row['name']} - ID Card Link: {id_card_link}")

            # Update the test_sheet with the new ID card link
            sheet.worksheet(TEST_SHEET_NAME).update_cell(index + 2, df.columns.get_loc('id_card') + 1, id_card_link)
            print(f"Updated {TEST_SHEET_NAME} for {row['name']} with new ID card link.")

    # Pause for 1 minute after processing each batch
    if batch_end < len(df):
        print("Batch processed. Pausing for 1 minute...")
        # time.sleep(60)

# Update folder details in the links sheet
existing_folder_names = links_sheet.col_values(1)
existing_folder_links = links_sheet.col_values(1)

# Update folder name
if 'folder_name' in existing_folder_names:
    folder_name_index = existing_folder_names.index('folder_name')
    links_sheet.update_cell(folder_name_index + 1, 2, folder_name)
else:
    links_sheet.append_row(['folder_name', folder_name])

# Update folder link
if 'id_card_folder_name' in existing_folder_names:
    id_card_folder_name_index = existing_folder_names.index('id_card_folder_name')
    links_sheet.update_cell(id_card_folder_name_index + 1, 2, id_card_folder_name)
else:
    links_sheet.append_row(['id_card_folder_name', id_card_folder_name])

if 'id_card_folder_link' in existing_folder_links:
    id_card_folder_link_index = existing_folder_links.index('id_card_folder_link')
    links_sheet.update_cell(id_card_folder_link_index + 1, 2, f"https://drive.google.com/drive/folders/{id_card_folder_id}")
else:
    links_sheet.append_row(['id_card_folder_link', f"https://drive.google.com/drive/folders/{id_card_folder_id}"])

# Wait for 1 minute before printing the final message
# time.sleep(60)
print("All operations completed successfully.")


In [ ]:
import requests
from io import BytesIO
from PIL import Image
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import time

# Connect to Google Drive and Sheets
creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/My Drive/my_private_files/mysheets.json',
    ['https://www.googleapis.com/auth/drive',
     'https://www.googleapis.com/auth/drive.file',
     'https://www.googleapis.com/auth/drive.metadata'])
client = gspread.authorize(creds)

# Open the Google Sheet by URL
sheet = client.open_by_url('https://docs.google.com')

# Assign sheet names
test_sheet = sheet.worksheet('COHORT')
links_sheet = sheet.worksheet('links')

# Check and possibly create a folder in Google Drive
folder_name = 'Photos'
drive_service = build('drive', 'v3', credentials=creds)
response = drive_service.files().list(q=f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'").execute()
if not response['files']:
    folder_metadata = {'name': folder_name, 'mimeType': 'application/vnd.google-apps.folder'}
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    folder_id = folder.get('id')
    drive_service.permissions().create(fileId=folder_id, body={'type': 'anyone', 'role': 'reader'}, fields='id').execute()
else:
    folder_id = response['files'][0]['id']

# Function to download image from Google Drive
def download_drive_file(file_id):
    request = drive_service.files().get_media(fileId=file_id)
    file_io = BytesIO()
    downloader = MediaIoBaseDownload(file_io, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
    file_io.seek(0)
    return Image.open(file_io)

# Function to download and validate image with compression
def download_and_validate_image(url, compress=True, max_size=(800, 600), quality=85):
    try:
        if 'drive.google.com' in url:
            file_id = url.split('id=')[1]
            image = download_drive_file(file_id)
        else:
            response = requests.get(url)
            image = Image.open(BytesIO(response.content))

        # Compression
        if compress:
            image.thumbnail(max_size)
            img_byte_arr = BytesIO()
            image.save(img_byte_arr, format='JPEG', quality=quality)
            img_byte_arr.seek(0)
            image = Image.open(img_byte_arr)

        if image.format not in ['JPEG', 'JPG', 'PNG']:
            return None, "Invalid format"
        return image, None
    except Exception as e:
        return None, str(e)

# Function to check if a file exists in the folder
def file_exists_in_folder(folder_id, file_name):
    query = f"name='{file_name}' and '{folder_id}' in parents"
    response = drive_service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
    files = response.get('files', [])
    return files

# Function to delete duplicate files
def delete_duplicates(folder_id, file_name):
    query = f"name='{file_name}' and '{folder_id}' in parents"
    response = drive_service.files().list(q=query, spaces='drive', fields='files(id, name, modifiedTime)').execute()
    files = response.get('files', [])
    if len(files) > 1:
        files.sort(key=lambda x: x['modifiedTime'])  # Sort files by modified time
        for file in files[:-1]:  # Delete all but the most recent file
            drive_service.files().delete(fileId=file['id']).execute()

# Process each row for passport photos
df = pd.DataFrame(test_sheet.get_all_records())
for index, row in df.iterrows():
    if pd.notna(row['photo_final']) and row['photo_final'].strip() != '':
        # Skip rows where photo_final column is already populated
        print(f"Row {index + 2} skipped: photo_final already contains a link.")
        continue

    if index > 0 and index % 25 == 0:
        print("Processed 25 records, pausing for a minute to avoid hitting API limit.")
        time.sleep(60)  # Pause for a minute

    processed = False
    for column_name in ['passportSizePic', 'backup_photo']:
        if column_name in df.columns and pd.notna(row[column_name]) and row[column_name].strip() != '':
            file_name = f'{row["name"]} [{row["iit_guwahati_id"]}] - Passport Photo.png'
            existing_files = file_exists_in_folder(folder_id, file_name)
            if existing_files:
                # Use existing link if file already exists
                file_link = f"https://drive.google.com/file/d/{existing_files[0]['id']}/view"
                test_sheet.update_cell(index + 2, df.columns.get_loc('photo_final') + 1, file_link)
                print(f"Row {index + 2} processed: existing file used.")
                processed = True
                break
            else:
                image, error = download_and_validate_image(row[column_name])
                if image:
                    img_byte_arr = BytesIO()
                    image.save(img_byte_arr, format='PNG')
                    img_byte_arr.seek(0)
                    media = MediaIoBaseUpload(img_byte_arr, mimetype='image/png', resumable=True)
                    file_metadata = {'name': file_name, 'parents': [folder_id]}
                    file = drive_service.files().create(body=file_metadata, media_body=media, fields='webViewLink').execute()
                    test_sheet.update_cell(index + 2, df.columns.get_loc('photo_final') + 1, file.get('webViewLink'))

                    # Delete duplicates
                    delete_duplicates(folder_id, file_name)

                    print(f"Row {index + 2} processed: new file created.")
                    processed = True
                    break
                else:
                    print(f"Failed to download image from {column_name} for {row['name']}, attempting to use backup.")
    if processed:
        # If the passport photo or backup photo was successfully processed, move to the next row
        continue

    # If neither passport photo nor backup photo was processed, update the cell with an empty string
    test_sheet.update_cell(index + 2, df.columns.get_loc('photo_final') + 1, "")
    print(f"Row {index + 2} processed: no valid photo found.")

print("All records processed. Pausing for a minute to ensure all operations are completed.")
# time.sleep(60)  # Additional pause to ensure all operations are completed

# Update folder details in the links sheet
existing_folder_names = links_sheet.col_values(1)
existing_folder_links = links_sheet.col_values(2)

# Check if folder name and link already exist, update if found, else append new rows
folder_name_row_index = None
folder_link_row_index = None

for i, name in enumerate(existing_folder_names):
    if name == 'folder_name':
        folder_name_row_index = i + 1
    elif name == folder_name:
        folder_name_row_index = i + 1

for i, link in enumerate(existing_folder_links):
    if link == 'folder_link':
        folder_link_row_index = i + 1
    elif link == f"https://drive.google.com/drive/folders/{folder_id}":
        folder_link_row_index = i + 1

if folder_name_row_index is not None:
    links_sheet.update_cell(folder_name_row_index, 2, folder_name)

if folder_link_row_index is not None:
    links_sheet.update_cell(folder_link_row_index, 2, f"https://drive.google.com/drive/folders/{folder_id}")

if folder_name_row_index is None:
    links_sheet.append_row(['folder_name', folder_name])

if folder_link_row_index is None:
    links_sheet.append_row(['folder_link', f"https://drive.google.com/drive/folders/{folder_id}"])

print("All operations completed successfully.")


In [ ]:
import time
from google.colab import drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from PIL import Image
from io import BytesIO

# Mount Google Drive and force remount if already mounted
drive.mount('/content/drive', force_remount=True)

# Setup Google Drive and Sheets
creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/My Drive/my_private_files/mysheets.json',
    ['https://www.googleapis.com/auth/drive']
)
client = gspread.authorize(creds)
sheet = client.open_by_url('https://docs.google.com')
test_sheet = sheet.worksheet('COHORT')
links_sheet = sheet.worksheet('links')

# Google Drive service
drive_service = build('drive', 'v3', credentials=creds)

# Function to check if a file exists and delete duplicates
def manage_files(folder_id, file_name):
    query = f"name='{file_name}' and '{folder_id}' in parents"
    response = drive_service.files().list(q=query, spaces='drive', fields='files(id, name, webViewLink, modifiedTime)').execute()
    files = response.get('files', [])
    if len(files) > 1:
        files.sort(key=lambda x: x['modifiedTime'], reverse=True)  # Keep the most recent file
        for file in files[1:]:  # Delete older duplicates
            drive_service.files().delete(fileId=file['id']).execute()
    return files[0]['webViewLink'] if files else None

# Ensure folder exists and handle folder link in links sheet
folder_name = 'ID_FINAL'
response = drive_service.files().list(q=f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'").execute()
folder_id = response['files'][0]['id'] if response['files'] else None
if not folder_id:
    folder_metadata = {'name': folder_name, 'mimeType': 'application/vnd.google-apps.folder'}
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    folder_id = folder.get('id')

# Update links in the Links sheet only if necessary
folder_link = f"https://drive.google.com/drive/folders/{folder_id}"
existing_links = links_sheet.get_all_records()
found = next((item for item in existing_links if item['Folder Name'] == folder_name), None)
if not found:
    links_sheet.append_row([folder_name, folder_link])

# Process records in batches
df = pd.DataFrame(test_sheet.get_all_records())
batch_size = 25
batches = [df[i:i + batch_size] for i in range(0, len(df), batch_size)]

for batch_index, batch in enumerate(batches):
    for index, row in batch.iterrows():
        id_card_url = row.get('id_card')
        photo_final_url = row.get('photo_final')
        final_id_link = row.get('id_card_final')

        if pd.notna(final_id_link) and final_id_link != '':
            print(f"Skipping {row['name']} as final ID link already exists.")
            continue

        if not id_card_url or not photo_final_url:
            print(f"Skipping {row['name']} due to missing ID card template or photo.")
            continue

        file_name = f"{row['name']}_{row['iit_guwahati_id']}_Final ID Card.jpg"
        existing_link = manage_files(folder_id, file_name)
        if existing_link:
            print(f"File already exists for {row['name']}. Updating link in sheet.")
            test_sheet.update_cell(index + 2, df.columns.get_loc('id_card_final') + 1, existing_link)
        else:
            try:
                print(f"Processing ID card for {row['name']}")

                # ID Card image processing
                id_card_file_id = id_card_url.split('/')[5]
                photo_final_file_id = photo_final_url.split('/')[5]

                id_card_request = drive_service.files().get_media(fileId=id_card_file_id)
                photo_final_request = drive_service.files().get_media(fileId=photo_final_file_id)

                id_card_fh = BytesIO()
                photo_final_fh = BytesIO()
                downloader_id_card = MediaIoBaseDownload(id_card_fh, id_card_request)
                downloader_photo_final = MediaIoBaseDownload(photo_final_fh, photo_final_request)

                done = False
                while not done:
                    status, done = downloader_id_card.next_chunk()
                    print(f"Downloading ID card template for {row['name']} - {int(status.progress() * 100)}% complete.")
                done = False
                while not done:
                    status, done = downloader_photo_final.next_chunk()
                    print(f"Downloading final photo for {row['name']} - {int(status.progress() * 100)}% complete.")

                id_card_fh.seek(0)
                photo_final_fh.seek(0)
                id_card_img = Image.open(id_card_fh)
                photo_final_img = Image.open(photo_final_fh)
                photo_final_img = photo_final_img.resize((200, 250))
                id_card_img.paste(photo_final_img, (40, 185))

                if id_card_img.mode != 'RGB':
                    id_card_img = id_card_img.convert('RGB')

                path_to_save = f"/content/drive/My Drive/{file_name}"
                id_card_img.save(path_to_save, format='JPEG')
                print(f"Saved final ID card for {row['name']} to {path_to_save}")

                file_metadata = {'name': file_name, 'parents': [folder_id]}
                media = MediaFileUpload(path_to_save, mimetype='image/jpeg')
                file = drive_service.files().create(body=file_metadata, media_body=media, fields='webViewLink').execute()
                print(f"Uploaded final ID card for {row['name']} with link {file['webViewLink']}")

                test_sheet.update_cell(index + 2, df.columns.get_loc('id_card_final') + 1, file['webViewLink'])
            except Exception as e:
                print(f"Error processing ID card for {row['name']}: {str(e)}")

    print(f"Batch {batch_index + 1} processed. Pausing for 1 minute.")
    #time.sleep(60)

# Final pause before ending
#time.sleep(60)
print("All operations completed successfully.")


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import requests
from datetime import datetime
import inflect
import time
import pytz

# Define sheet names
TEST_SHEET_NAME = 'COHORT'
LINKS_SHEET_NAME = 'links'

# Google Drive and Google Sheets setup
creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/My Drive/my_private_files/mysheets',
    ['https://www.googleapis.com/auth/drive',
     'https://www.googleapis.com/auth/drive.file',
     'https://www.googleapis.com/auth/drive.metadata']
)
client = gspread.authorize(creds)
sheet = client.open_by_url('https://docs.google.com/spreadsheets')

# Assign sheet names
test_sheet = sheet.worksheet(TEST_SHEET_NAME)
links_sheet = sheet.worksheet(LINKS_SHEET_NAME)

# Load custom font from URL
font_url = 'https://github.com/google/fonts/raw/main/ofl/poppins/Poppins-Regular.ttf'
response = requests.get(font_url)
font_file = BytesIO(response.content)
font = ImageFont.truetype(font_file, 30)

# Load the registration fee invoice template
reg_fee_template_path = '/content/drive/My Drive/IIGS-FILES/templates/reg_fee.png'
reg_fee_template = Image.open(reg_fee_template_path)

# Check for the existence of the "Reg Fee Invoice" folder on Google Drive
drive_service = build('drive', 'v3', credentials=creds)
folder_name = 'Reg Fee Invoice'
response = drive_service.files().list(q=f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'").execute()
if not response['files']:
    folder_metadata = {'name': folder_name, 'mimeType': 'application/vnd.google-apps.folder'}
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    folder_id = folder.get('id')
else:
    folder_id = response['files'][0]['id']

# Initialize the inflect engine
p = inflect.engine()

# Get all records from the test sheet
records = test_sheet.get_all_records()

# Batch size for processing records
batch_size = 25

# Process records in batches
for batch_start in range(0, len(records), batch_size):
    batch_end = min(batch_start + batch_size, len(records))
    batch_records = records[batch_start:batch_end]

    for index, row in enumerate(batch_records, start=batch_start):
        # Check if the 'reg_fee_invoice' link is already present
        reg_fee_invoice_link = row.get('reg_fee_invoice', '')
        if reg_fee_invoice_link and reg_fee_invoice_link.lower() not in ['nan', 'null', '']:
            continue  # Skip record if the link is already present

        # Check if the 'reg_fee_dictionary' column is filled
        reg_fee_dict_str = row.get('reg_fee_dictionary', '')
        if not reg_fee_dict_str:
            test_sheet.update_cell(index + 2, test_sheet.find("reg_fee_invoice").col, "")
            # time.sleep(3)
            continue  # Skip record if the 'reg_fee_dictionary' field is missing

        # Parse the 'reg_fee_dictionary' column
        try:
            reg_fee_dict = dict(
                line.split(': ', 1) for line in reg_fee_dict_str.split('\n') if ': ' in line
            )
        except ValueError as e:
            print(f"Error parsing reg_fee_dictionary for row {index + 2}: {e}")
            test_sheet.update_cell(index + 2, test_sheet.find("reg_fee_invoice").col, "")
            # time.sleep(3)
            continue  # Skip record if there was an error parsing the dictionary

        required_keys = ['name', 'iit_guwahati_id', 'course_name', '5k_payment_time', '5k_order_id', 'reg_fee_particulars', 'reg_fee_amount']
        if not all(key in reg_fee_dict for key in required_keys):
            test_sheet.update_cell(index + 2, test_sheet.find("reg_fee_invoice").col, "")
            # time.sleep(3)
            continue  # Skip record if any required key is missing in the parsed dictionary

        # Check if any values are blank, NaN, or null
        if any(not reg_fee_dict[key] or reg_fee_dict[key].lower() in ['nan', 'null'] or reg_fee_dict[key].isspace() for key in required_keys):
            test_sheet.update_cell(index + 2, test_sheet.find("reg_fee_invoice").col, "")
           # time.sleep(3)
            continue  # Skip record if any value is blank, NaN, or null

        invoice_filename = f"{reg_fee_dict['name']} [{reg_fee_dict['iit_guwahati_id']}] - Registration Fee Receipt.pdf"
        existing_files = drive_service.files().list(q=f"name='{invoice_filename}' and '{folder_id}' in parents", fields='files(id, webViewLink)').execute().get('files', [])
        if existing_files:
            # File already exists, update the spreadsheet with existing file link
            existing_file = existing_files[0]
            file_link = existing_file.get('webViewLink')
            test_sheet.update_cell(index + 2, test_sheet.find("reg_fee_invoice").col, file_link)
        else:
            # Create a new invoice image from the template
            invoice_image = reg_fee_template.copy()
            invoice_draw = ImageDraw.Draw(invoice_image)

            # Drawing the information on the invoice
            invoice_draw.text((365, 450), reg_fee_dict['name'], fill='black', font=font)
            invoice_draw.text((365, 520), reg_fee_dict['iit_guwahati_id'], fill='black', font=font)
            invoice_draw.text((365, 590), reg_fee_dict['course_name'], fill='black', font=font)
            invoice_draw.text((1300, 450), reg_fee_dict['5k_payment_time'], fill='black', font=font)
            invoice_draw.text((1300, 520), reg_fee_dict['5k_order_id'], fill='black', font=font)
            invoice_draw.text((1300, 585), "Online", fill='black', font=font)
            invoice_draw.text((110, 790), reg_fee_dict['reg_fee_particulars'], fill='black', font=font)
            invoice_draw.text((1980, 790), reg_fee_dict['reg_fee_amount'], fill='black', font=font)
            invoice_draw.text((1980, 860), reg_fee_dict['reg_fee_amount'], font=font, fill='black')

            # Convert the amount to words
            reg_fee_amount_numeric = int(float(reg_fee_dict['reg_fee_amount'].replace('₹', '').replace(',', '').strip()))
            amount_in_text = p.number_to_words(reg_fee_amount_numeric).capitalize() + " rupees only/-"
            invoice_draw.text((420, 985), amount_in_text, font=font, fill='black')

            # Get the current time in IST
            ist = pytz.timezone('Asia/Kolkata')
            current_time_ist = datetime.now(ist)
            formatted_time_ist = current_time_ist.strftime("%Y-%m-%d %H:%M:%S")
            invoice_draw.text((420, 1050), formatted_time_ist, font=font, fill='black')

            # Convert the image from RGBA to RGB
            rgb_invoice_image = invoice_image.convert('RGB')

            # Save the invoice as a PDF
            invoice_image_path = f'/tmp/{invoice_filename}'
            rgb_invoice_image.save(invoice_image_path, 'PDF')

            # Upload the invoice to Google Drive
            invoice_file_metadata = {'name': invoice_filename, 'parents': [folder_id]}
            invoice_media = MediaFileUpload(invoice_image_path, mimetype='application/pdf')
            invoice_file = drive_service.files().create(body=invoice_file_metadata, media_body=invoice_media, fields='webViewLink').execute()
            invoice_link = invoice_file.get('webViewLink')
            test_sheet.update_cell(index + 2, test_sheet.find("reg_fee_invoice").col, invoice_link)

    # Pause for a minute after processing each batch
    print(f"Batch {batch_start // batch_size + 1} completed. Pausing for a minute...")
   # time.sleep(60)

# Ensure the 'links' sheet handles folder names and links correctly
existing_folder_names = links_sheet.col_values(1)
existing_folder_links = links_sheet.col_values(2)

# Check if headers are present and add if not
if not existing_folder_names or existing_folder_names[0].lower() != 'folder name':
    links_sheet.insert_row(['Folder Name', 'Folder Link'], 1)
    existing_folder_names.insert(0, 'Folder Name')
    existing_folder_links.insert(0, 'Folder Link')

# Check and update or append folder information
folder_link = f"https://drive.google.com/drive/folders/{folder_id}"
folder_name_index = None

for index, name in enumerate(existing_folder_names):
    if name == folder_name:
        folder_name_index = index + 1  # +1 because spreadsheet indices start at 1

if folder_name_index is not None:
    links_sheet.update_cell(folder_name_index, 2, folder_link)
else:
    links_sheet.append_row([folder_name, folder_link])

# Final pause before printing completion message
print("All batches processed. Pausing for a minute before finalizing...")
# time.sleep(60)

# Get the current time in IST for the final print statement
current_time_ist_final = datetime.now(ist).strftime("%Y-%m-%d %H:%M:%S")
print(f"All operations completed successfully at {current_time_ist_final} IST.")


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import requests
from datetime import datetime
import pytz
import inflect
import time
import math
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define sheet names
TEST_SHEET_NAME = 'COHORT'
LINKS_SHEET_NAME = 'links'

print("Setting up Google Drive and Google Sheets credentials...")
# Google Drive and Google Sheets setup
creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/drive/My Drive/my_private_files/mysheets.json',
    ['https://www.googleapis.com/auth/drive',
     'https://www.googleapis.com/auth/drive.file',
     'https://www.googleapis.com/auth/drive.metadata']
)
client = gspread.authorize(creds)
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1u1tnJwZ0xDjPH0BxX5wN-94VJV86bCVkjSZlc2M5334/edit#gid=1527167846')

# Assign sheet names
test_sheet = sheet.worksheet(TEST_SHEET_NAME)
links_sheet = sheet.worksheet(LINKS_SHEET_NAME)

print("Loading custom font from URL...")
# Load custom font from URL
font_url = 'https://github.com/google/fonts/raw/main/ofl/poppins/Poppins-Regular.ttf'
response = requests.get(font_url)
font_file = BytesIO(response.content)
font = ImageFont.truetype(font_file, 30)

print("Loading the registration fee invoice template...")
# Load the registration fee invoice template
reg_fee_template_path = '/content/drive/My Drive/IIGS-FILES/templates/trimerster_I_fee.png'
reg_fee_template = Image.open(reg_fee_template_path)

print("Checking for the existence of the 'Trimester Fee Invoice' folder on Google Drive...")
# Check for the existence of the "Trimester Fee Invoice" folder on Google Drive
drive_service = build('drive', 'v3', credentials=creds)
folder_name = 'Trimester Fee Invoice'
response = drive_service.files().list(q=f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'").execute()
if not response['files']:
    folder_metadata = {'name': folder_name, 'mimeType': 'application/vnd.google-apps.folder'}
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    folder_id = folder.get('id')
    print(f"Created new folder with ID: {folder_id}")
else:
    folder_id = response['files'][0]['id']
    print(f"Found existing folder with ID: {folder_id}")

# Initialize the inflect engine
p = inflect.engine()

print("Getting all records from the test sheet...")
# Get all records from the test sheet
records = test_sheet.get_all_records()

# Batch processing with a delay between batches
batch_size = 50
num_batches = math.ceil(len(records) / batch_size)

for batch_num in range(num_batches):
    print(f"Processing batch {batch_num + 1} of {num_batches}...")
    batch_records = records[batch_num * batch_size:(batch_num + 1) * batch_size]

    for index, row in enumerate(batch_records, start=batch_num * batch_size):
        print(f"Processing record {index + 1}...")

        # Check if the 'trimester_fee_invoice' link is already present
        trimester_fee_invoice_link = row.get('trimester_fee_invoice', '')
        if trimester_fee_invoice_link and trimester_fee_invoice_link.lower() not in ['nan', 'null', '']:
            continue  # Skip record if the link is already present

        # Check if the 'trimester_fee_dictionary' column is filled
        reg_fee_dict_str = row.get('trimester_fee_dictionary', '')
        if not reg_fee_dict_str:
            test_sheet.update_cell(index + 2, test_sheet.find("trimester_fee_invoice").col, "")
            print("Skipped record: 'trimester_fee_dictionary' field is missing.")
            continue  # Skip record if the 'trimester_fee_dictionary' field is missing

        # Parse the 'trimester_fee_dictionary' column
        reg_fee_dict = dict(line.split(': ') for line in reg_fee_dict_str.split('\n'))

        required_keys = ['name', 'iit_guwahati_id', 'course_name', '60k_payment_time', '60k_order_id', 'trimester_fee_particulars', 'trimester_fee_amount']
        if not all(key in reg_fee_dict for key in required_keys):
            test_sheet.update_cell(index + 2, test_sheet.find("trimester_fee_invoice").col, "")
            print("Skipped record: Missing required keys in the parsed dictionary.")
            continue  # Skip record if any required key is missing in the parsed dictionary

        # Check if any values are blank, NaN, or null
        if any(not reg_fee_dict[key] or reg_fee_dict[key].lower() in ['nan', 'null'] or reg_fee_dict[key].isspace() for key in required_keys):
            test_sheet.update_cell(index + 2, test_sheet.find("trimester_fee_invoice").col, "")
            print("Skipped record: One or more required values are blank, NaN, or null.")
            continue  # Skip record if any value is blank, NaN, or null

        invoice_filename = f"{reg_fee_dict['name']} [{reg_fee_dict['iit_guwahati_id']}] - Trimester Fee Receipt.pdf"
        existing_files = drive_service.files().list(q=f"name='{invoice_filename}' and '{folder_id}' in parents", fields='files(id, webViewLink)').execute().get('files', [])
        if existing_files:
            # File already exists, update the spreadsheet with existing file link
            existing_file = existing_files[0]
            file_link = existing_file.get('webViewLink')
            test_sheet.update_cell(index + 2, test_sheet.find("trimester_fee_invoice").col, file_link)
            print(f"Updated spreadsheet with existing file link for: {invoice_filename}")
        else:
            # Create a new invoice image from the template
            invoice_image = reg_fee_template.copy()
            invoice_draw = ImageDraw.Draw(invoice_image)

            # Drawing the information on the invoice
            invoice_draw.text((365, 450), reg_fee_dict['name'], fill='black', font=font)
            invoice_draw.text((365, 520), reg_fee_dict['iit_guwahati_id'], fill='black', font=font)
            invoice_draw.text((365, 590), reg_fee_dict['course_name'], fill='black', font=font)
            invoice_draw.text((1300, 450), reg_fee_dict['60k_payment_time'], fill='black', font=font)
            invoice_draw.text((1300, 520), reg_fee_dict['60k_order_id'], fill='black', font=font)
            invoice_draw.text((1300, 585), "Upfront-Direct", fill='black', font=font)
            invoice_draw.text((110, 790), reg_fee_dict['trimester_fee_particulars'], fill='black', font=font)
            invoice_draw.text((1955, 790), reg_fee_dict['trimester_fee_amount'], fill='black', font=font)
            invoice_draw.text((1955, 860), reg_fee_dict['trimester_fee_amount'], font=font, fill='black')

            # Convert the amount to words
            reg_fee_amount_numeric = int(float(reg_fee_dict['trimester_fee_amount'].replace('₹', '').replace(',', '').strip()))
            amount_in_text = p.number_to_words(reg_fee_amount_numeric).capitalize() + " rupees only/-"
            invoice_draw.text((420, 985), amount_in_text, font=font, fill='black')

            # Get current time in Indian Standard Time (IST)
            ist = pytz.timezone('Asia/Kolkata')
            current_time = datetime.now(ist).strftime("%Y-%m-%d %H:%M:%S")
            invoice_draw.text((420, 1050), current_time, font=font, fill='black')

            # Convert the image from RGBA to RGB
            rgb_invoice_image = invoice_image.convert('RGB')

            # Save the invoice as a PDF
            invoice_image_path = f'/tmp/{invoice_filename}'
            rgb_invoice_image.save(invoice_image_path, 'PDF')

            print(f"Uploading new invoice to Google Drive: {invoice_filename}")
            # Upload the invoice to Google Drive
            invoice_file_metadata = {'name': invoice_filename, 'parents': [folder_id]}
            invoice_media = MediaFileUpload(invoice_image_path, mimetype='application/pdf')
            invoice_file = drive_service.files().create(body=invoice_file_metadata, media_body=invoice_media, fields='webViewLink').execute()
            invoice_link = invoice_file.get('webViewLink')
            test_sheet.update_cell(index + 2, test_sheet.find("trimester_fee_invoice").col, invoice_link)
            print(f"Uploaded and updated spreadsheet with new file link for: {invoice_filename}")

    # Delay between batches
    print("Batch processing complete. Waiting for 1 minute before processing the next batch...")
    time.sleep(60)

# Ensure the 'links' sheet handles folder names and links correctly
print("Ensuring the 'links' sheet handles folder names and links correctly...")
existing_folder_names = links_sheet.col_values(1)
existing_folder_links = links_sheet.col_values(2)

# Check if headers are present and add if not
if not existing_folder_names or existing_folder_names[0].lower() != 'folder name':
    links_sheet.insert_row(['Folder Name', 'Folder Link'], 1)
    existing_folder_names.insert(0, 'Folder Name')
    existing_folder_links.insert(0, 'Folder Link')
    print("Added headers to 'links' sheet.")

# Check and update or append folder information
folder_link = f"https://drive.google.com/drive/folders/{folder_id}"
folder_name_index = None

for index, name in enumerate(existing_folder_names):
    if name == folder_name:
        folder_name_index = index + 1  # +1 because spreadsheet indices start at 1

if folder_name_index is not None:
    links_sheet.update_cell(folder_name_index, 2, folder_link)
    print(f"Updated 'links' sheet with folder link for: {folder_name}")
else:
    links_sheet.append_row([folder_name, folder_link])
    print(f"Appended new folder link to 'links' sheet for: {folder_name}")

# Final delay before completion
print("All batches processed. Waiting for 1 minute before finalizing...")
#time.sleep(60)

print("All operations completed successfully.")


In [ ]:
# from PIL import Image
# from io import BytesIO
# import pandas as pd
# import gspread
# from googleapiclient.discovery import build
# from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
# from oauth2client.service_account import ServiceAccountCredentials
# from tempfile import NamedTemporaryFile
# import os
# import time

# # Mount Google Drive (if using Colab)
# from google.colab import drive
# drive.mount('/content/drive')

# # File path
# json_keyfile_path = '/content/drive/My Drive/my_private_files/mysheets.json'

# # Check if the JSON keyfile exists
# if not os.path.exists(json_keyfile_path):
#     raise FileNotFoundError(f"The file {json_keyfile_path} does not exist. Please check the path.")

# # Initialize Google Drive and Sheets API
# creds = ServiceAccountCredentials.from_json_keyfile_name(
#     json_keyfile_path,
#     ['https://www.googleapis.com/auth/drive']
# )
# drive_service = build('drive', 'v3', credentials=creds)
# client = gspread.authorize(creds)

# # Continue with the rest of your code
# sheet = client.open_by_url('https://docs.google.com/=sharing')
# test_sheet = sheet.worksheet('test_sheetv2')
# links_sheet = sheet.worksheet('links')

# # Load the template
# template_path = '/content/drive/My Drive/IIGS-FILES/templates/Stationary-Kit-Sales.png'
# template_img = Image.open(template_path)

# # Ensure "share_kit" folder exists
# folder_name = 'share_kit'
# query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'"
# response = drive_service.files().list(q=query).execute()
# folder_id = response['files'][0]['id'] if response['files'] else None
# if not folder_id:
#     folder_metadata = {'name': folder_name, 'mimeType': 'application/vnd.google-apps.folder'}
#     folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
#     folder_id = folder.get('id')

# # Set desired sizes and positions
# admission_letter_size = (700, 950)
# admission_letter_position = (1025, 120)
# id_card_final_size = (600, 360)
# id_card_final_position = (200, 120)

# def download_file(file_id):
#     """Download a file from Google Drive given a file ID."""
#     request = drive_service.files().get_media(fileId=file_id)
#     fh = BytesIO()
#     downloader = MediaIoBaseDownload(fh, request)
#     done = False
#     while not done:
#         status, done = downloader.next_chunk()
#     fh.seek(0)
#     return fh

# # Processing records
# df = pd.DataFrame(test_sheet.get_all_records())

# # Batch size
# batch_size = 25

# # Iterate through the records in batches
# for batch_start in range(0, len(df), batch_size):
#     batch_end = min(batch_start + batch_size, len(df))
#     cell_updates = []

#     for index, row in df.iloc[batch_start:batch_end].iterrows():
#         student_name = row['name']
#         student_code = row['iit_guwahati_id']
#         combined_filename = f"{student_name} [{student_code}]_Combined.png"
#         combined_file_link = row['combined_file']

#         # Skip if the link is already present and not blank/NaN/null
#         if pd.notna(combined_file_link) and combined_file_link.strip():
#             print(f"Link already present for {student_name}: {combined_file_link}")
#             continue

#         # Skip if admission letter or ID card final is not present
#         if not row['admission_letter'].strip() or not row['id_card_final'].strip():
#             print(f"Skipping {student_name} due to missing admission letter or ID card.")
#             continue

#         # Check if combined file already exists
#         query = f"name='{combined_filename}' and '{folder_id}' in parents"
#         existing_files = drive_service.files().list(q=query, fields='files(id, webViewLink)').execute().get('files', [])

#         if existing_files:
#             webViewLink = existing_files[0]['webViewLink']
#             print(f"Existing file found for {student_name}: {webViewLink}")
#         else:
#             # Download and process images
#             try:
#                 admission_letter_id = row['admission_letter'].split('/')[5]
#                 id_card_final_id = row['id_card_final'].split('/')[5]
#             except IndexError:
#                 print(f"Skipping {student_name} due to malformed URL.")
#                 continue

#             admission_letter_fh = download_file(admission_letter_id)
#             id_card_final_fh = download_file(id_card_final_id)

#             admission_letter_img = Image.open(admission_letter_fh).resize(admission_letter_size)
#             id_card_final_img = Image.open(id_card_final_fh).resize(id_card_final_size)

#             # Paste images on the template
#             template_with_images = template_img.copy()
#             template_with_images.paste(admission_letter_img, admission_letter_position, mask=admission_letter_img if admission_letter_img.mode == 'RGBA' else None)
#             template_with_images.paste(id_card_final_img, id_card_final_position, mask=id_card_final_img if id_card_final_img.mode == 'RGBA' else None)

#             # Save the combined image to a temporary file and upload
#             with NamedTemporaryFile(delete=False, suffix='.png') as tmp:
#                 template_with_images.save(tmp.name, format='PNG')
#                 tmp.close()
#                 file_metadata = {'name': combined_filename, 'parents': [folder_id]}
#                 media = MediaFileUpload(tmp.name, mimetype='image/png')
#                 file = drive_service.files().create(body=file_metadata, media_body=media, fields='webViewLink').execute()
#                 webViewLink = file.get('webViewLink')
#                 print(f"New file created for {student_name}: {webViewLink}")

#         # Collect the cell update
#         cell = test_sheet.cell(index + 2, df.columns.get_loc('combined_file') + 1)
#         cell.value = webViewLink
#         cell_updates.append(cell)

#         # To avoid hitting the rate limit, introduce a small delay
#         time.sleep(1)

#     # Perform the cell updates if there are any
#     if cell_updates:
#         test_sheet.update_cells(cell_updates)

#     # Pause for a minute between batches
#     # time.sleep(60)

# # Update links sheet for the folder
# folder_link = f"https://drive.google.com/drive/folders/{folder_id}"
# existing_links = links_sheet.get_all_records()
# found = next((item for item in existing_links if item['Folder Name'] == folder_name), None)
# if not found:
#     links_sheet.append_row([folder_name, folder_link])

# # Pause for a minute before printing the final completion statement
# # time.sleep(60)

# print("All operations completed successfully.")
